In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Hayvan listesi
animal_classes = [
    "antelope", "badger", "bat", "bear", "bee", "beetle", "bison", "boar", "butterfly", "cat",
    "caterpillar", "chimpanzee", "cockroach", "cow", "coyote", "crab", "crow", "deer", "dog", "dolphin",
    "donkey", "dragonfly", "duck", "eagle", "elephant", "flamingo", "fly", "fox", "goat", "goldfish",
    "goose", "gorilla", "grasshopper", "hamster", "hare", "hedgehog", "hippopotamus", "hornbill", "horse", "hummingbird",
    "hyena", "jellyfish", "kangaroo", "koala", "ladybugs", "leopard", "lion", "lizard", "lobster", "mosquito",
    "moth", "mouse", "octopus", "okapi", "orangutan", "otter", "owl", "ox", "oyster", "panda",
    "parrot", "pelecaniformes", "penguin", "pig", "pigeon", "porcupine", "possum", "raccoon", "rat", "reindeer",
    "rhinoceros", "sandpiper", "seahorse", "seal", "shark", "sheep", "snake", "sparrow", "squid", "squirrel",
    "starfish", "swan", "tiger", "turkey", "turtle", "whale", "wolf", "wombat", "woodpecker", "zebra"
]

# Görüntüleri yüklemek için dataset
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.data = ImageFolder(root_dir, transform=transform)
        self.classes = self.data.classes

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
# Ön işleme ve augmentation
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(224, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Veri seti yolu
data_dir = r"C:\Users\dulek\Downloads\yazlab3\data\train"
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Veri seti dizini '{data_dir}' bulunamadı. Lütfen doğru yolu belirtin veya veri setini bu konuma yerleştirin.")

# Veri setini yükleme
dataset = CustomImageDataset(root_dir=data_dir, transform=data_transforms)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [7]:
# Swin Transformer Tiny'yi yükleme
model = models.swin_t(weights=models.Swin_T_Weights.IMAGENET1K_V1)
num_ftrs = model.head.in_features
model.head = nn.Linear(num_ftrs, len(dataset.classes))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
early_stopping_patience = 5
best_val_loss = float('inf')
epochs_no_improve = 0

In [9]:
num_epochs = 10
train_losses, val_losses = [], []
train_accs, val_accs = [], []

# Model ve metriklerin kaydedileceği dizin
save_dir = r"C:\Users\dulek\OneDrive\Belgeler\Python\MultiZoo"
os.makedirs(save_dir, exist_ok=True)

for epoch in range(num_epochs):
    # Eğitim
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Doğrulama
    model.eval()
    val_running_loss = 0.0
    correct = 0
    total = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss = val_running_loss / len(val_loader)
    val_acc = correct / total
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    # Erken durdurma
    scheduler.step(val_loss)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), os.path.join(save_dir, 'best_model.pth'))
        print(f"Saved best model to {os.path.join(save_dir, 'best_model.pth')}")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= early_stopping_patience:
            print("Early stopping triggered")
            break

# Son modeli kaydet
torch.save(model.state_dict(), os.path.join(save_dir, 'final_model.pth'))
print(f"Saved final model to {os.path.join(save_dir, 'final_model.pth')}")

# Eğitim metriklerini kaydet
metrics = {
    'train_losses': train_losses,
    'val_losses': val_losses,
    'train_accs': train_accs,
    'val_accs': val_accs
}
with open(os.path.join(save_dir, 'training_metrics.pkl'), 'wb') as f:
    pickle.dump(metrics, f)
print(f"Saved training metrics to {os.path.join(save_dir, 'training_metrics.pkl')}")

Epoch 1/10, Train Loss: 0.2939, Train Acc: 0.9471, Val Loss: 0.2796, Val Acc: 0.9172, Precision: 0.9275, Recall: 0.9172, F1: 0.9169
Saved best model to C:\Users\dulek\OneDrive\Belgeler\Python\MultiZoo\best_model.pth
Epoch 2/10, Train Loss: 0.1763, Train Acc: 0.9701, Val Loss: 0.2791, Val Acc: 0.9130, Precision: 0.9275, Recall: 0.9130, F1: 0.9130
Saved best model to C:\Users\dulek\OneDrive\Belgeler\Python\MultiZoo\best_model.pth
Epoch 3/10, Train Loss: 0.1230, Train Acc: 0.9809, Val Loss: 0.2444, Val Acc: 0.9266, Precision: 0.9337, Recall: 0.9266, F1: 0.9267
Saved best model to C:\Users\dulek\OneDrive\Belgeler\Python\MultiZoo\best_model.pth
Epoch 4/10, Train Loss: 0.0997, Train Acc: 0.9858, Val Loss: 0.2480, Val Acc: 0.9235, Precision: 0.9320, Recall: 0.9235, F1: 0.9230
Epoch 5/10, Train Loss: 0.0787, Train Acc: 0.9895, Val Loss: 0.2508, Val Acc: 0.9361, Precision: 0.9429, Recall: 0.9361, F1: 0.9361
Epoch 6/10, Train Loss: 0.0653, Train Acc: 0.9911, Val Loss: 0.2563, Val Acc: 0.9340, Pr